### Database with 4*4 images

shapes : square, triangle, cirle
colors : light red, light blue, light yellow, light green

dataset size : 2,000,000

probability to have a symbol at each position : 0.5

Same as v0.1.2 but with 2M images and with making sure there isn't any duplicates while generating the DB.



In [1]:
import os

db_dir = os.environ["DATA"] + "PatImgXAI_data/db0.1.3/"
os.makedirs(db_dir, exist_ok=True)

In [2]:
from xaipatimg.datagen.dbimg import load_db

db = load_db(db_dir)

In [3]:
# Number of images generated
NBGEN = 2000000

# Grid division of each image
X_DIVISIONS = 4
Y_DIVISIONS = 4

# Size of the images in pixels
img_size = (700, 700)

# Probability to generate a geometrical shape at each position in the grid
SHAPE_PROB = 0.5

# Define available shapes
SHAPES = ['circle', 'square', 'triangle']
COLORS  = ["#F86C62", "#7AB0CD", "#F4D67B", "#87C09C"]

In [4]:
import numpy as np
from xaipatimg.datagen.dbimg import generate_uuid
import os

to_generate = NBGEN
unique_content_generated = {}
duplicate_count = 0
while to_generate > 0:
    content = []
    for i in range(X_DIVISIONS):
        for j in range(Y_DIVISIONS):
            if np.random.random() < SHAPE_PROB:
                content.append({
                    "shape": np.random.choice(SHAPES),
                    "pos": (i, j),
                    "color": np.random.choice(COLORS)
                })

    if str(content) in unique_content_generated:
        duplicate_count += 1
        continue

    imgid = generate_uuid()
    db[imgid] = {
        "path": os.path.join("img", imgid + ".png"),
        "division" : (X_DIVISIONS, Y_DIVISIONS),
        "size": img_size,
        "content": content
    }

    unique_content_generated[str(content)] = True
    to_generate -= 1

print("unique generated in DB : " + str(len(db)))
print("duplicates avoided : " + str(duplicate_count))

unique generated in DB : 2000000
duplicates avoided : 930


## Validating no duplicate is found

In [5]:
import tqdm

content_dict = {}
nb_duplicates = 0

for k, v in tqdm.tqdm(db.items()):
    if str(v["content"]) in content_dict:
        nb_duplicates += 1
    else:
        content_dict[str(v["content"])] = True

print(nb_duplicates)

100%|██████████| 2000000/2000000 [00:35<00:00, 55882.50it/s]

0


In [6]:
from xaipatimg.datagen.genimg import gen_img_and_save_db
gen_img_and_save_db(db, db_dir, overwrite=False, n_jobs=190)

100%|██████████| 2000000/2000000 [28:55<00:00, 1152.59it/s] 


### Create dataset with blue diagonal rule

In [3]:
def blue_diagonal_rule(img_content):
    symbols_in_diagonal = 0
    for c in img_content:
        if c["pos"][0] == c["pos"][1]:
            symbols_in_diagonal += 1
            if c["color"] != "#7AB0CD":
                return False
    return symbols_in_diagonal >= 1

In [4]:
from xaipatimg.datagen.gendataset import create_dataset_based_on_rule
import os
csv_name_train = "bluediagonal_train.csv"
csv_name_test = "bluediagonal_test.csv"
csv_name_valid = "bluediagonal_valid.csv"
sample_img_path = os.path.join(db_dir, "datasets", "bluediagonal_train")
create_dataset_based_on_rule(db_dir, csv_name_train, csv_name_test, csv_name_valid, test_size=4000, valid_size=4000,
                             dataset_pos_samples_nb=12000, dataset_neg_samples_nb=12000, rule_fun=blue_diagonal_rule)

100%|██████████| 2000000/2000000 [00:03<00:00, 546962.75it/s]


In [6]:
from xaipatimg.datagen.gendataset import extract_sample_from_dataset
extract_sample_from_dataset(db_dir, csv_name_train, sample_img_path, 1000, 1000)

16001it [00:00, 50915.16it/s]


### Create dataset with exactly four squares rule

In [7]:
def four_squares_total(img_content):
    sq_total = 0
    for c in img_content:
        if c["shape"] == "square":
            sq_total += 1
    return sq_total == 4

In [9]:
from xaipatimg.datagen.gendataset import create_dataset_based_on_rule
import os
csv_name_train = "foursquares_train.csv"
csv_name_test = "foursquares_test.csv"
csv_name_valid = "foursquares_valid.csv"
sample_img_path = os.path.join(db_dir, "datasets", "foursquares_train")

create_dataset_based_on_rule(db_dir, csv_name_train, csv_name_test, csv_name_valid, test_size=4000, valid_size=4000,
                             dataset_pos_samples_nb=12000, dataset_neg_samples_nb=12000, rule_fun=four_squares_total)

100%|██████████| 2000000/2000000 [00:03<00:00, 521408.78it/s]


In [10]:
from xaipatimg.datagen.gendataset import extract_sample_from_dataset
extract_sample_from_dataset(db_dir, csv_name_train, output_dir_path=sample_img_path, pos_samples_nb=1000, neg_samples_nb=1000)

16001it [00:01, 11662.01it/s]


### Create dataset with exactly three squares rule


In [11]:
def three_squares_total(img_content):
    sq_total = 0
    for c in img_content:
        if c["shape"] == "square":
            sq_total += 1
    return sq_total == 3

In [13]:
csv_name_train = "threesquares_train.csv"
csv_name_test = "threesquares_test.csv"
csv_name_valid = "threesquares_valid.csv"
sample_img_path = os.path.join(db_dir, "datasets", "threesquares_train")

create_dataset_based_on_rule(db_dir, csv_name_train, csv_name_test, csv_name_valid, test_size=4000, valid_size=4000,
                             dataset_pos_samples_nb=12000, dataset_neg_samples_nb=12000, rule_fun=three_squares_total)

100%|██████████| 2000000/2000000 [00:03<00:00, 581294.30it/s]


In [14]:
from xaipatimg.datagen.gendataset import extract_sample_from_dataset
extract_sample_from_dataset(db_dir, csv_name_train, output_dir_path=sample_img_path, pos_samples_nb=1000, neg_samples_nb=1000)

16001it [00:01, 11705.87it/s]


### Create dataset with any image rule (to show sample of the full database)

In [16]:
def any_image(img_content):
    return True

from xaipatimg.datagen.gendataset import create_dataset_based_on_rule
import os
csv_name_train = "any_train.csv"
csv_name_test = "any_test.csv"
csv_name_valid = "any_valid.csv"
sample_img_path = os.path.join(db_dir, "datasets", "any_sample")

create_dataset_based_on_rule(db_dir, csv_name_train, csv_name_test, csv_name_valid, test_size=4000, valid_size=4000,
                             dataset_pos_samples_nb=12000, dataset_neg_samples_nb=0, rule_fun=any_image)

100%|██████████| 2000000/2000000 [00:01<00:00, 1840013.76it/s]


In [17]:
from xaipatimg.datagen.gendataset import extract_sample_from_dataset
extract_sample_from_dataset(db_dir, csv_name_train, output_dir_path=sample_img_path, pos_samples_nb=1000, neg_samples_nb=0)

4001it [00:00, 7695.82it/s]


### Create dataset "twice as many red symbols as blue symbols"

In [21]:
def twice_as_many_red_as_blue(img_content):
    blue_total = 0
    red_total = 0

    for c in img_content:
        if c["color"] == "#F86C62":
            red_total += 1
        elif c["color"] == "#7AB0CD":
            blue_total += 1

    return red_total == 2 * blue_total and blue_total != 0

In [22]:
import os
from xaipatimg.datagen.gendataset import create_dataset_based_on_rule

csv_name_train = "twiceasmanyredasblue_train.csv"
csv_name_test = "twiceasmanyredasblue_test.csv"
csv_name_valid = "twiceasmanyredasblue_valid.csv"
sample_img_path = os.path.join(db_dir, "datasets", "twiceasmanyredasblue_train")

create_dataset_based_on_rule(db_dir, csv_name_train, csv_name_test, csv_name_valid, test_size=4000, valid_size=4000,
                             dataset_pos_samples_nb=12000, dataset_neg_samples_nb=12000, rule_fun=twice_as_many_red_as_blue)

100%|██████████| 2000000/2000000 [00:04<00:00, 452148.82it/s]


In [23]:
from xaipatimg.datagen.gendataset import extract_sample_from_dataset
extract_sample_from_dataset(db_dir, csv_name_train, output_dir_path=sample_img_path, pos_samples_nb=1000, neg_samples_nb=1000)

16001it [00:01, 12776.29it/s]
